### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

In [2]:
!pip install --upgrade --quiet  lark qdrant-client

### Load CSV data from S3

In [3]:
!pwd

/home/ec2-user/SageMaker/Enterprise-RAG/notebooks


In [4]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,CookTime_Minutes,PrepTime_Minutes,TotalTime_Minutes,Combined_Features_Clean
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",1440,45,1485,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...,25,240,265,Biryani Chicken Breast Make share Biryani reci...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu...",5,30,35,Best Lemonade Beverages one first Good House K...
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces...",20,1440,1460,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,...",30,20,50,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'CookTime_Minutes', 'PrepTime_Minutes', 'TotalTime_Minutes',
       'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

'Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy'

In [8]:
df = df[df['AggregatedRating'].notna()]
df.shape

(269294, 32)

### Load data into chunked documents

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [10]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Note: chunking may not be useful for us as the EDA has the following about description length
print(new_recipe['Combined_Features_Clean'].str.len().describe())


count    522517.000000\
mean        211.461774\
std         122.388646\
min          44.000000\
25%         131.000000\
50%         180.000000\
75%         255.000000\
max        4174.000000\
Name: Combined_Features_Clean, dtype: float64

In [10]:
def create_textsplitter(chunks, overlaps):
    splits = {}
    for chunk in chunks:
        for overlap in overlaps:
            splits[f'chunk{str(chunk)}_overlap{str(overlap)}'] = RecursiveCharacterTextSplitter(chunk_size=chunk, chunk_overlap=overlap)
    return splits

In [11]:
chunks = [128, 256, 512, 1024, 2048]
overlaps = [.25, .2, .15, .1]

text_splits = create_textsplitter(chunks, overlaps)

In [12]:
text_splits['chunk1024_overlap0.25']

In [11]:
def create_documents(df):
    documents = []
    for index, row in df.iterrows():
        metadata = {
            'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
            'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
            'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
            'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
            'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
            'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
            'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
            'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
            'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
            'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
            'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
            'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
            'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
            'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
            'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
            'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
            'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
            'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
            'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
        }

        # Use Combined_Features_Clean for the document content
        text = str(row['Combined_Features_Clean'])
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)
        
    return documents


In [12]:
# Take a sample of 100K receipes 
documents = create_documents(df.sample(n=10000))

In [13]:
documents[0]

Document(metadata={'recipe_id': '170223.0', 'name': 'Thirty-Minute Muesli', 'cook_time': 'No Cook Time Available', 'prep_time': 'PT10M', 'total_time': 'PT10M', 'recipe_category': 'Breakfast', 'keywords': "['Lunch/Snacks' 'Grains' 'Apple' 'Pears' 'Fruit' 'Swiss' 'European'\n 'Low Cholesterol' 'Healthy' '< 15 Mins' 'No Cook' 'Refrigerator'\n 'Beginner Cook' 'Easy' 'Inexpensive']", 'aggregated_rating': '5.0', 'review_count': '10.0', 'calories': '199.5', 'fat_content': '6.5', 'saturated_fat_content': '1.2', 'cholesterol_content': '3.7', 'sodium_content': '70.9', 'carbohydrate_content': '31.6', 'sugar_content': '13.9', 'protein_content': '8.7', 'recipe_servings': '6.0', 'recipe_yield': '3 cups'}, page_content='Thirty-Minute Muesli Breakfast think easy breakfast snack sounds yummy healthful. Plan half-hour refrigeration time. UCSD Healthy Diet Diabetes submitted Zaar World Tour II. Lunch/Snacks Grains Apple Pears Fruit Swiss European Low Cholesterol Healthy < 15 Mins Cook Refrigerator Beginn

In [14]:
len(documents)

10000

In [17]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [18]:
split_documents = {key: split_documents_with_metadata(documents, text_splitter_recursive) for key, text_splitter_recursive in text_splits.items()}

In [19]:
split_documents['chunk256_overlap0.25'][0]

Document(metadata={'recipe_id': '137828.0', 'name': 'Coney Island Chili Sauce for Hot Dogs', 'cook_time': 'PT15M', 'prep_time': 'PT10M', 'total_time': 'PT25M', 'recipe_category': 'Sauces', 'keywords': "['< 30 Mins' 'Stove Top' 'Easy']", 'aggregated_rating': '5.0', 'review_count': '14.0', 'calories': '141.7', 'fat_content': '8.7', 'saturated_fat_content': '3.4', 'cholesterol_content': '38.6', 'sodium_content': '343.4', 'carbohydrate_content': '4.4', 'sugar_content': '2.9', 'protein_content': '11.5', 'recipe_servings': '4.0', 'recipe_yield': 'No Yield Information Available', 'chunk_id': 0}, page_content='Coney Island Chili Sauce Hot Dogs Sauces Make share Coney Island Chili Sauce Hot Dogs recipe Food.com. < 30 Mins Stove Top Easy')

In [15]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

# From documents with no chunking
qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

# from split documents (specify the chunk and overlap key)
# qdrant_store = Qdrant.from_documents(split_documents['chunk1024_overlap0.25'],
#     embedding_model,
#     location=":memory:",
# )

## Create llm query call

In [43]:
import json

In [44]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [26]:
baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [47]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


In [27]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [56]:
qa_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

You are evaluating for consistency of the user query.
Please retrieve documents that DO NOT violate dietary restrictions, allergies, 
or any requirements dictated by the user.

"""

In [55]:
def qa_query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': qa_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            # "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

## Create Retrieval Chainfrom langchain_core.prompts import ChatPromptTemplate

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [31]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

In [65]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [66]:
def process_prompt_basic(query_args):
    prompt_with_context = baseline_user_prompt.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

## Create Initial Retriever

In [34]:
qdrant_retriever_rerank = qdrant_store.as_retriever(search_type='mmr', search_kwargs={"k": 50, 'lambda_mult': 0.5})


### Self Query

In [60]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = metadata_field_info = [
    AttributeInfo(
        name="keywords",
        description="Keywords within the recipe",
        type="string or list[string]",
    )
]

document_content_description = "List of recipes"
llm = qa_query_bedrock_llm
qa_retriever = SelfQueryRetriever.from_llm(
    llm, qdrant_store, document_content_description, metadata_field_info, verbose=True
)

In [62]:
qa_retriever.invoke("Recipes with no peanuts")

TypeError: Object of type StringPromptValue is not JSON serializable

### Cross encoder query

In [38]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=
)

compressed_docs = compression_retriever.invoke("What pizza can I make in 20 minutes?")
compressed_docs

[Document(metadata={'recipe_id': '371206.0', 'name': 'Mexican Pizza', 'cook_time': 'PT10M', 'prep_time': 'PT10M', 'total_time': 'PT20M', 'recipe_category': '< 30 Mins', 'keywords': "['Easy']", 'aggregated_rating': '5.0', 'review_count': '8.0', 'calories': '1368.4', 'fat_content': '128.7', 'saturated_fat_content': '23.8', 'cholesterol_content': '67.0', 'sodium_content': '632.6', 'carbohydrate_content': '34.2', 'sugar_content': '2.6', 'protein_content': '23.5', 'recipe_servings': '2.0', 'recipe_yield': 'No Yield Information Available', '_id': '816101324f9c406e8484dae4bbce8842', '_collection_name': '7437dba00d064c55a4ae2fde63943c52'}, page_content='Mexican Pizza < 30 Mins found recipe online www.mexicanfoodrecipes.org posting ZWT. recipe says use seasoned ground beef give directions/ingredients seasoning. suggest using Recipe #369923 using package taco seasoning. Please feel free serve pizza salsa, guacamole, lettuce topping choice. recipe two servings feel free double, triple etc. amount

### Chain

In [69]:
qdrant_rag_chain_rerank = (
    RunnableMap(
        {"context": qdrant_retriever_rerank | qa_retriever | format_docs,
         "query": RunnablePassthrough()}
    )
    | process_prompt_basic
    | query_bedrock_llm
)

In [70]:
qdrant_rag_chain_rerank.invoke("Give me a peanut free Thai dish")

TypeError: Object of type StringPromptValue is not JSON serializable